In [1]:
!pip install torch torchvision pydicom numpy opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [30]:
import os
import numpy as np
import pydicom
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
from scipy.ndimage import zoom
import random
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
def load_dicom_slices(dicom_folder):
    slices = []
    for dicom_file in sorted(glob.glob(os.path.join(dicom_folder, "*.dcm"))):
        ds = pydicom.dcmread(dicom_file)
        slices.append(ds.pixel_array)
    return np.stack(slices, axis=-1)

dicom_folder = '/content/drive/MyDrive/IAAA-Challenge/data/*'
dicom_folders = glob.glob(dicom_folder)[:100]
labels_df = pd.read_csv("/content/drive/MyDrive/IAAA-Challenge/train.csv")

# Example DICOM folders for 10 patients
data = []
labels = []

for dicom_folder in dicom_folders:
    image_3d = load_dicom_slices(dicom_folder)
    data.append(image_3d)
    label = labels_df[labels_df["SeriesInstanceUID"]==dicom_folder.split("/")[-1]]
    labels.append(label["prediction"].values[0])
    print(len(labels))

np.save('/content/drive/MyDrive/IAAA-Challenge/brain_mri_data_multiple_patients.npy', {'data': data, 'labels': labels})

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
class BrainMRIDataset(Dataset):
    def __init__(self, numpy_file_path,target_size=(128, 128, 16),transform=None):
        data_dict = np.load(numpy_file_path,allow_pickle=True).item()
        self.data = data_dict['data']
        self.labels = data_dict['labels']
        self.target_size = target_size
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx].astype(np.float32)  # Convert to float32
        image = self.resize_image(image, self.target_size)  # Resize the image
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        # Add channel dimension
        image = np.expand_dims(image, axis=0)
        return image, label

    def resize_image(self, image, target_size):
        factors = (
            target_size[0] / image.shape[0],
            target_size[1] / image.shape[1],
            target_size[2] / image.shape[2]
        )
        return zoom(image, factors, order=1)

# Example usage
numpy_file_path = 'brain_mri_data_multiple_patients.npy'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = BrainMRIDataset(numpy_file_path,target_size=(128, 128, 16), transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [38]:
# Step 3: Build and train a 3D ResNet model from scratch
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(planes)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet3D(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet3D, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv3d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def ResNet18_3D(num_classes):
    return ResNet3D(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = ResNet18_3D(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')


Epoch [1/10], Step [10/25], Loss: 0.0111
Epoch [1/10], Step [20/25], Loss: 0.0530
Epoch [2/10], Step [10/25], Loss: 1.4774
Epoch [2/10], Step [20/25], Loss: 0.0670
Epoch [3/10], Step [10/25], Loss: 0.0583
Epoch [3/10], Step [20/25], Loss: 0.5564
Epoch [4/10], Step [10/25], Loss: 0.0564
Epoch [4/10], Step [20/25], Loss: 0.1001
Epoch [5/10], Step [10/25], Loss: 0.0401
Epoch [5/10], Step [20/25], Loss: 0.0689
Epoch [6/10], Step [10/25], Loss: 0.0799
Epoch [6/10], Step [20/25], Loss: 0.0649
Epoch [7/10], Step [10/25], Loss: 0.7695
Epoch [7/10], Step [20/25], Loss: 0.7215
Epoch [8/10], Step [10/25], Loss: 0.0703
Epoch [8/10], Step [20/25], Loss: 0.0897
Epoch [9/10], Step [10/25], Loss: 0.6595
Epoch [9/10], Step [20/25], Loss: 0.1686
Epoch [10/10], Step [10/25], Loss: 1.5197
Epoch [10/10], Step [20/25], Loss: 0.5491
